In [5]:
import json
import pandas as pd
import geopandas as gpd
import geopy
from flask import Flask, render_template, request
import folium
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [6]:
from shapely.geometry import Point

In [8]:
# Load the Excel files
df1 = pd.read_excel("Delhi_Devices_Aug_Dec_2022.xlsx", sheet_name="temp")
df2 = pd.read_excel("Delhi_Devices_Jan_July_2022.xlsx", sheet_name="temp")

# Merge the two data frames
df = pd.concat([df1, df2], ignore_index=True)

# Remove any rows where the `station_name` column is null or not a valid geometry object
for i in range(len(df)):
    try:
        geometry = df.loc[i, "station"]
        gpd.points_from_xy([geometry.x, geometry.y])
    except (ValueError, AttributeError):
        df.drop(index=i, inplace=True)

# Override the default user agent
geopy.geocoders.options.default_user_agent = "my-application"

# Create a geocoder object
geocoder = geopy.Nominatim(user_agent="my-application-name")

# Create a GeoDataFrame from the `station` column
gdf = gpd.GeoDataFrame(df, geometry=df["station"])

# Add latitude and longitude columns to the GeoDataFrame
gdf["latitude"] = gdf["geometry"].apply(lambda x: geocoder.geocode(x).latitude)
gdf["longitude"] = gdf["geometry"].apply(lambda x: geocoder.geocode(x).longitude)

In [22]:
import pyarrow.parquet

In [23]:
# Save the GeoDataFrame
gdf.to_parquet("combined_data_with_lat_long.parquet")